In [3]:
import streamlit as st
import folium
from streamlit_folium import st_folium
import pandas as pd
from io import BytesIO
from pyproj import Proj, Transformer
import pandas as pd
def geodetic_to_utm(lat, lon):
    """Converte coordenadas geodésicas (lat, lon) para UTM."""
    # Determinar o fuso UTM baseado na longitude
    utm_zone = int((lon + 180) / 6) + 1
    proj_string = f"+proj=utm +zone={utm_zone} +datum=WGS84 +units=m +no_defs"
    
    # Criar um transformador
    transformer = Transformer.from_crs("EPSG:4326", proj_string, always_xy=True)
    
    # Converter coordenadas
    easting, northing = transformer.transform(lon, lat)
    return utm_zone, easting, northing
df=pd.read_excel('poligonais.xlsx')
df['num_no']=range(1,len(df)+1)

df=df[['num_no','Tipo','Latitude','Longitude']]

df[["UTM_Zone", "x", "y"]] = df.apply(lambda row: geodetic_to_utm(row["Latitude"], row["Longitude"]), axis=1, result_type="expand")
df['num_no']=range(1,len(df)+1)
df["UTM_Zone"]=df["UTM_Zone"].astype(int)

df_rio=df[df['Tipo']=='Rio'].reset_index(drop=True)
fim_rio=df_rio.at[df_rio.index[-1],'num_no']
gmsh=[]
for i in range(1,fim_rio+1):
    no='Point('+f'{i}'+')={'+f'{df_rio.at[i-1,'x']}'+','+f'{df_rio.at[i-1,'y']}'+',0};'
    gmsh.append(no)
loop='{'
for i in range(1,fim_rio+1):
    linha='Line('+f'{i}'+')={'+f'{i}'+','+f'{i+1 if i+1!=fim_rio+1 else 1}'+'};'
    gmsh.append(linha)
    loop=loop + f'{i},'
loop=loop[:-1]
gmsh.append('Curve Loop(1) ='+loop +'};')

qtd_ilhas=len(df['Tipo'].unique())-1
loop2='{1,'
for ilha in range(1,qtd_ilhas+1):#
    nome_ilha=f'Ilha_{ilha}'
    df_ilha=df[df['Tipo']==nome_ilha].reset_index(drop=True)
    for i in range(df_ilha.at[0,'num_no'],df_ilha.at[df_ilha.index[-1],'num_no']+1):
        no='Point('+f'{i}'+')={'+f'{df.at[i-1,'x']}'+','+f'{df.at[i-1,'y']}'+',0};'
        gmsh.append(no)
    loop='{'
    for i in range(df_ilha.at[0,'num_no'],df_ilha.at[df_ilha.index[-1],'num_no']+1):
        linha='Line('+f'{i}'+')={'+f'{i}'+','+f'{i+1 if i+1!=df_ilha.at[df_ilha.index[-1],'num_no']+1 else df_ilha.at[0,'num_no']}'+'};'
        gmsh.append(linha)
        loop=loop + f'{i},'
    loop=loop[:-1]
    gmsh.append(f'Curve Loop({ilha+1}) ='+loop +'};')
    loop2=loop2 + f'{-ilha-1},'

loop2=loop2[:-1]
gmsh.append('Plane Surface(1) = '+loop2 +'};')

GMSH=pd.DataFrame({'aa':gmsh})
GMSH.to_csv("GMSH.txt", sep="\t", index=False, header=False)

In [61]:
df

,num_no,Tipo,Latitude,Longitude,UTM_Zone,x,y
0,1,Rio,-14.434680,-53.833008,22,194535.335184,-1.597684e+06
1,2,Rio,-14.432685,-53.804684,22,197588.805803,-1.597426e+06
2,3,Rio,-14.442244,-53.818502,22,196110.751995,-1.598502e+06
3,4,Ilha_1,-14.435345,-53.829017,22,194966.887924,-1.597753e+06
4,5,Ilha_1,-14.435013,-53.826528,22,195235.008178,-1.597712e+06
5,6,Ilha_1,-14.436447,-53.826892,22,195197.602487,-1.597872e+06
6,7,Ilha_2,-14.435013,-53.824725,22,195429.492288,-1.597710e+06
7,8,Ilha_2,-14.434982,-53.822848,22,195632.037200,-1.597704e+06
8,9,Ilha_2,-14.436218,-53.822622,22,195658.030256,-1.597841e+06
9,10,Ilha_2,-14.436488,-53.823448,22,195569.260114,-1.597872e+06


In [67]:
aa=None+12
aa

# Aplicar conversão em todas as linhas


TypeError: unsupported operand type(s) for +: 'NoneType' and 'int'

In [43]:
GMSH

,aa
0,"Point(1)={194535.33518402395,-1597684.27652075..."
1,"Point(2)={197588.8058033124,-1597425.895814555,0}"
2,"Point(3)={196110.75199529732,-1598502.47262510..."
